In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

In [2]:
from PIL import Image
path='data/train'

images=[]
pacientes=[]
list_directorios=[folder[1] for folder in os.walk(path)]
for paciente in list_directorios[0][:100]:
    fotos_paciente=[folde[2] for folde in os.walk(f'{path}/{paciente}/FLAIR')]
    if len(fotos_paciente)==0:
        continue
    for foto in fotos_paciente[0]:
        images.append(Image.open(f"{path}/{paciente}/FLAIR/{foto}").convert('RGB'))
        pacientes.append(paciente)
        
        

In [3]:
# Creo un df con el id del paciente y la fotos
# (cada fila es una foto, un paciente tiene muchas filas por lo tanto)

#TRAIN
images_df=pd.DataFrame()
images_df['Pacientes']=pacientes
images_df['Imagenes']=images
images_df

,Pacientes,Imagenes
0,00658,<PIL.Image.Image image mode=RGB size=512x512 a...
1,00658,<PIL.Image.Image image mode=RGB size=512x512 a...
2,00658,<PIL.Image.Image image mode=RGB size=512x512 a...
3,00658,<PIL.Image.Image image mode=RGB size=512x512 a...
4,00658,<PIL.Image.Image image mode=RGB size=512x512 a...
...,...,...
9354,00725,<PIL.Image.Image image mode=RGB size=512x512 a...
9355,00725,<PIL.Image.Image image mode=RGB size=512x512 a...
9356,00725,<PIL.Image.Image image mode=RGB size=512x512 a...
9357,00725,<PIL.Image.Image image mode=RGB size=512x512 a...


In [4]:
# Import labels
labels_df=pd.read_csv('train_labels.csv')
labels_df=labels_df.rename(columns={'BraTS21ID': 'Pacientes'})
images_df['Pacientes']=pd.to_numeric(images_df['Pacientes'])

#  Añado las labels con left join
images_df=images_df.merge(labels_df,on='Pacientes',how='left')
# images_df=images_df.drop(columns=['Pacientes','%Blanco']).rename(columns={'MGMT_value':'Label'})


#Genero train/test set

train_df=images_df.iloc[:6000]
test_df=images_df.iloc[6000:]

In [117]:
#Hago los trai/test sets de diferentes tamaños

# Train 882 Test 378
train_small_df=train_df.sample(frac=0.03,random_state=200).reset_index().drop(columns=['index','Pacientes'])
test_small_df=test_df.sample(frac=0.03,random_state=200).reset_index().drop(columns=['index','Pacientes'])

# Train 2940 Test 1260
train_medium_df=train_df.sample(frac=0.1,random_state=200).reset_index().drop(columns=['index','Pacientes'])
test_medium_df=test_df.sample(frac=0.1,random_state=200).reset_index().drop(columns=['index','Pacientes'])

# Train 8820 Test 2781
train_big_df=train_df.sample(frac=0.3,random_state=200).reset_index().drop(columns=['index','Pacientes'])
test_big_df=test_df.sample(frac=0.3,random_state=200).reset_index().drop(columns=['index','Pacientes'])

In [302]:
import torch
#Creo clase para facilitar la entrada de los datos a dataloaders
from PIL import Image
import torchvision.transforms as T

class BrainData(torch.utils.data.Dataset):
    def __init__(self, dataframe,):
       
     
        #self.transforms = transforms
        self.dataframe=dataframe
        

    def __getitem__(self, idx):
         
        #Unnecesary
        #pacient_id =self.dataframe.loc[[idx], ['Pacientes']]
        #Get image and category, the two statements that vit is going to use to learn
        imgs=[x[0] for x in self.dataframe.loc[[idx],['Imagenes']].values][0]
        imgs=T.ToTensor()(imgs)
        imgs=T.ConvertImageDtype(torch.float)(imgs)
        imgs=T.Resize((224,224))(imgs)
        imgs=T.CenterCrop(224)(imgs)
        imgs=T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5,0.5,0.5])(imgs)
        
        #Cuando los labels esten en el df
        category=int(self.dataframe.loc[[idx],['MGMT_value']].values)
        if category==0:
            category=torch.Tensor([0.])
        else:
            category=torch.Tensor([1.])
        
        #if self.transforms is not None:
            #imgs = self.transforms(imgs)

        return imgs, category

    def __len__(self):
        return len(self.dataframe['MGMT_value'])

In [232]:
#Transforms: Main, the resize to the vit default size. Maybe we can append more???
import torchvision
import torchvision.transforms as T

import random

def get_transform(train=True):
    transform = []
    transform.append(T.PILToTensor())
    transform.append(T.ConvertImageDtype(torch.float))
    transform.append(T.Resize((224,224)))
 

    return T.Compose(transform)

In [314]:
#Creation of datasets (one to train, one to test)

train_dataset=BrainData(train_small_df)
test_dataset=BrainData(test_small_df)

In [315]:

train_dataset[0]

(tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]),
 tensor([0.]))

In [316]:
tensor1=torch.tensor([[-0.001],[0.01]])

torch.round(torch.sigmoid(tensor1))
    


tensor([[0.],
        [1.]])

In [317]:
#Dataloaders

import torch.utils

#data_loaders
data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=32, shuffle=True, num_workers=0)

data_loader_test = torch.utils.data.DataLoader(
        test_dataset, batch_size=32, shuffle=False, num_workers=0)

dataloaders={"train": data_loader, "test": data_loader_test}

dataset_sizes={"train": len(train_dataset), "test":len(test_dataset)}

In [318]:
#Train model function

import time
import copy
import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_model(model, dataloaders, criterion, optimizer, scheduler, num_epochs=25):
    model.to(device)
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'test']:
            if phase == 'train':
                model.train()  # Set model to training mode
                scheduler.step()
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            #Tqdm for estimation of time
            for inputs, labels in tqdm.tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
               
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = torch.round(torch.sigmoid(outputs))
                    loss = criterion(torch.reshape(outputs,(1,-1)), torch.reshape(labels.data,(1,-1)))
                
               
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                print(preds)
               #Doesnt matter, only if we have time
                #running_loss += loss.item() * inputs.size(0)
              
                running_corrects += torch.sum(torch.reshape(preds,(1,-1)) == torch.reshape(labels.data,(1,-1)))
              
             #Doesnt matter, only if we have time
            #epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects/ dataset_sizes[phase]

            print(f'{phase} Acc: {epoch_acc:.4f}')

            # Save best model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best test Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [319]:
#Define the parameters
import torchvision
from torch import nn, optim
from torch.optim import lr_scheduler
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#Load VIT (Maybe a big vit model)
model_transf = torchvision.models.vit_b_16(pretrained=True)

for param in model_transf.parameters():
    param.requires_grad = False




#Number of classes: 2, positive or negative. Access to the last layer and change output size
model_transf.heads = nn.Linear( model_transf.heads.head.in_features, 1,)

#loss function
criterion = nn.CrossEntropyLoss()
optimizer= optim.SGD(model_transf.parameters(), lr=0.001, momentum=0.9)
lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

/Users/alvarodelamaza/.pyenv/versions/lewagon/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/alvarodelamaza/.pyenv/versions/lewagon/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [320]:

#Train time!!

print(device)
model_transf = train_model(model_transf, dataloaders, criterion, optimizer,lr_scheduler, num_epochs=25)

/Users/alvarodelamaza/.pyenv/versions/lewagon/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


cpu
Epoch 0/24
----------


 17%|█▋        | 1/6 [00:13<01:05, 13.00s/it]

tensor([[1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.]], grad_fn=<RoundBackward0>)


 33%|███▎      | 2/6 [00:26<00:52, 13.21s/it]

...

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.]])


 75%|███████▌  | 3/4 [00:38<00:12, 12.82s/it]

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]])


100%|██████████| 4/4 [00:40<00:00, 10.15s/it]

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.]])
test Acc: 0.5149

Training complete in 42m 5s
Best test Acc: 0.633663


In [328]:
torch.save(model_transf,'models/model_1')

: 